In [2]:
# !pip install pyarrow
import pandas as pd
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [3]:
DATA_DIR = '../../data/'
http_bots = pq.read_table(DATA_DIR + 'botsHTTPRequests-20180216_1416GMT.parquet').to_pandas()
http_users = pq.read_table(DATA_DIR + 'usersHTTPRequests-20180216_1416GMT.parquet').to_pandas()
#http_bots = http_bots.sample(frac = 0.03)
#http_users = http_users.sample(frac=0.03)
http_users['target'] = 0
http_bots['target'] = 1

http = pd.concat([http_bots, http_users])

http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId,hour,target
0,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518791752182,None,https://ok.ru/web-api/pts/postingFormJs,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,0hs35/dB/t+uT7u/c71+Vw==,17,1
1,8.293748e+18,friendMain,3rRnET9ABG3feVMANINXpg==,POST,friendAltGroup,https://www.ok.ru/,NAV,1518793194690,friendAltGroup,https://www.ok.ru/profile/561514539517/groups,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17,1
2,7.895490e+17,None,Qsv9DECmtHgfUZ0GAVUoLQ==,POST,PinsEditAjaxRB,https://ok.ru/,REQ,1518791913138,None,https://ok.ru/profile/589603942951/statuses,Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537....,e9iJCewg5Ww1ijKmYKdAyg==,17,1
3,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518792629926,None,https://ok.ru/web-api/pts/webpush,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17,1
4,-8.322615e+18,None,XK5THOHUVJHpqGojQAKrMQ==,GET,userMain,None,NAV,1518795472623,userMain,https://ok.ru/,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,f3Z8Sr501bB3SvEgCo+9Rg==,18,1


In [14]:
http.shape

(1010377, 14)

In [4]:
http = http.sample(frac=1).reset_index(drop=True)

In [6]:
# Сюда вклиниваются None езли что
print('Кол-во уникальных операций: {}'.format(len(http['operation'].unique())))

Кол-во уникальных операций: 480


### Приведем категориальный столбей общей выборки к разряженному виду и разделим выборку на train и testh

Сначала закодируем названия операций чиселками

In [7]:
operations = http['operation']

dict_code_op = {}

c = 0
for op in operations.unique():
    dict_code_op[op] = c
    c += 1
    
operations = operations.apply(lambda x: dict_code_op[x])    
#operations = pd.DataFrame()

In [8]:
operations_sparse = csr_matrix(([1] * operations.shape[0],
                                operations,
                                range(0, operations.shape[0] + 1, 1)))[:, 1:]
#     http.drop('target', axis=1), http['target'], test_size=0.3, random_state=17)
#operations_sparse.toarray()

#### Разделим выборку на train и holdout

In [9]:
# operations_sparse -> X_train/X_test; http.target -> y_train/y_test
idx_split = int(http.shape[0] * 0.7)
X_train = operations_sparse[:idx_split]
X_holdout = operations_sparse[idx_split:] #X_test = operations_sparse[idx_split:, :] для df
y_train = http['target'][:idx_split]
y_holdout = http['target'][idx_split:]

#### Обучим первую модель

In [10]:
%%time
logit = LogisticRegression(C=0.1, n_jobs=-1, random_state=17)
logit.fit(X_train, y_train)

/home/greg/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 1.68 s, sys: 39.7 ms, total: 1.72 s
Wall time: 1.76 s


In [17]:
holdout_pred = logit.predict_proba(X_holdout)[:, 1]
holdout_pred.shape

(303114,)

In [19]:
roc_auc_score(y_holdout, holdout_pred)

0.9184732600156248

In [49]:
pred_holdout_series = pd.Series(holdout_pred).apply(lambda x: 1 if x>=0.3 else 0);

In [50]:
print(metrics.classification_report(y_holdout, pred_holdout_series))

             precision    recall  f1-score   support

          0       0.99      1.00      1.00    299945
          1       0.71      0.21      0.32      3169

avg / total       0.99      0.99      0.99    303114



#### Добавим еще один признак

In [ ]:
http['url'].value_counts()

In [ ]:
http['url'].notna().shape[0] / http.shape[0]

In [ ]:
# Сделает словарик всех операций
def make_dic(col):
    dict = {}
    count = 0
    for v in col.unique():
        dict[v] = count
        count += 1
    
    return dict

encoded_urls = make_dic(http['url'])
http['url'] = http['url'].apply(lambda x: encoded_urls[x])